# Visualization

A Dash app is made to view the 2- and 3-D embeddings interactively.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

import numpy as np
import pandas as pd

Big thanks to Kevin Mader at [this repo](https://github.com/4QuantOSS/DashIntro) for the function below to show Dash apps within Jupyter. Very handy for development.

In [2]:
from IPython import display
def show_app(app,  # type: dash.Dash
             port=9999,
             width= 1000,
             height=750,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    
    Author: Kevin Mader @ https://github.com/4QuantOSS/DashIntro
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)

In [ ]:
app = dash.Dash()

df = pd.read_table('data/all.tsv', sep='\t', encoding='utf-8')
embedding_name_map = {
    'PCA': 'Principal Component Analysis',
    'MDS': 'Multidimensional Scaling',
    'SE': 'Spectral Embedding',
    'TSNE': 't-distributed Stochastic Neighbor Embedding',
    'ISOMAP': 'Isomap'
}
embeddings = list(embedding_name_map.keys())
pairs = [ # (Congress, Session)
    (115, 2017),
    (114, 2016),
    (114, 2015),
    (113, 2014),
    (113, 2013),
    (112, 2012),
    (112, 2011),
    (111, 2010),
    (111, 2009)
]
slider_map = {
    2009: '2009',
    2010: '',
    2011: '2011',
    2012: '',
    2013: '2013',
    2014: '',
    2015: '2015',
    2016: '',
    2017: '2017',
}


congress_map = dict((j, i) for i,j in pairs)
colormap = {'Democrat':'rgba(0, 0, 255, 0.5)', 'Republican':'rgba(255, 0, 0, 0.5)', 'Other': 'rgba(0, 255, 0, 0.5)'}
chambermap = {'sen': 'Senator', 'rep': 'Representative'}

base_layout = go.Layout(
    legend = {'orientation': 'h', 'x': .00, 'y': 1.05},
    paper_bgcolor='#F2F2F2',
    plot_bgcolor='#F2F2F2'
)

dropdown_title_style = {'background-color': '#FFFFFF', 'border-spacing': 0, 'border-collapse': 'separate', 'box-sizing': 'border-box'}

app.layout = html.Div(
    [
        html.Div(
            [
                html.Div(
                    [
                        html.Span('Embedding'),
                        dcc.Dropdown(
                            id = 'embedding-select',
                            options = [{'label': embedding, 'value': embedding} for embedding in embeddings],
                            value = embeddings[0]
                        )
                    ],
                    style = {'width': '48%', 'display': 'inline-block', 'margin': 'auto 5px auto auto'}
                ),
                html.Div(
                    [
                        html.Span('Dimensions', style = {'text-align': 'center'}),
                        dcc.Dropdown(
                            id = 'components-select',
                            options = [
                                {'label': '1 & 2', 'value': '1,2'},
                                {'label': '1 & 3', 'value': '1,3'},
                                {'label': '2 & 3', 'value': '2,3'},
                                {'label': '3D', 'value': '3D'}
                            ],
                            value = '1,2'
                        )
                    ],
                    style = {'width': '48%', 'display': 'inline-block', 'margin': 'auto auto auto 5px'}
                ),
            ], 
            style = {'width': '40%', 'margin': 'auto'}
        ),
        html.Div(
            [
                html.Div(
                    [html.Span('Year')],
                    style = {'width': '15%', 'display': 'inline-block'}
                ),
                html.Div(
                    [
                        dcc.Slider(
                            id = 'session-select',
                            min = min(congress_map.keys()),
                            max = max(congress_map.keys()),
                            step = 1,
                            value = max(congress_map.keys()),
                            included = False,
                            updatemode = 'drag',
                            marks = slider_map
                        )
                    ],
                    style = {'width': '85%', 'display': 'inline-block'}
                )
            ],
            style = {'width': '40%', 'margin': '15px auto auto auto', 'height': '35px'}
            
        ),
        dcc.Graph(id='embeddings-graph', style = {'height': '550px', 'width': '100%'})
    ], 
    style = {'background-color': '#F2F2F2', 'width': '100%', 'max-width': '1024px'}
)

@app.callback(
    dash.dependencies.Output('embeddings-graph', 'figure'),
    [
        dash.dependencies.Input('session-select', 'value'),
        dash.dependencies.Input('embedding-select', 'value'),
        dash.dependencies.Input('components-select', 'value')
    ]
)
def update_graph(session, embedding, components_pair):
    data = []
    congress = congress_map[session]
    session = str(session)
    df_selection = df[(df['congress'] == int(congress)) & (df['session'] == int(session))]
    
    base_layout.update({'title': 'Congress {}, Session {}<br>{}'.format(congress, session, embedding_name_map[embedding])})
    
    if components_pair != '3D':
    
        components = components_pair.split(',')
        c1, c2 = embedding+str(components[0]), embedding+str(components[1])


        for party, party_color in colormap.items():
            if party == 'Other':
                df_p = df_selection.loc[~df['party'+session].isin(['Democrat', 'Republican']),:]
            else:
                df_p = df_selection.loc[df['party'+session] == party,:]
            hover_text = df_p['name.official_full']+'<br>'+df_p['chamber'+session].map(chambermap)+'<br>'+df_p['state'+session]

            trace = go.Scatter(
                x = df_p[c1],
                y = df_p[c2],
                text = hover_text,
                name = party,
                mode = 'markers',
                hoverinfo = ('text'),
                marker = {
                    'size': 6,
                    'color': party_color,
                    'line': {'width': 1}
                }
            )

            data.append(trace)

        layout = go.Layout(
            hovermode = 'closest',
            xaxis = {'title': c1, 'showline': True, 'zeroline': False},
            yaxis = {'title': c2, 'showline': True, 'zeroline': False},
        )
        
    else:
        
        c1, c2, c3 = embedding+'1', embedding+'2', embedding+'3'
        
        for party, party_color in colormap.items(): 
            party_color = party_color.replace('0.5', '1') # Any transparency looks terrible in 3D
            if party == 'Other':
                df_p = df_selection.loc[~df['party'+session].isin(['Democrat', 'Republican']),:]
            else:
                df_p = df_selection.loc[df['party'+session] == party,:]
            hover_text = df_p['name.official_full']+'<br>'+df_p['chamber'+session].map(chambermap)+'<br>'+df_p['state'+session] 

            trace = go.Scatter3d(
                x = df_p[c1],
                y = df_p[c2],
                z = df_p[c3],
                text = hover_text,
                name = party,
                mode = 'markers',
                hoverinfo = ('text'),
                marker = {
                    'size': 4,
                    'color': party_color,
                    'line': {'width': 1, 'color': 'rgba(0, 0, 0, 0.85)'}
                }
            )
            data.append(trace)

        layout = go.Layout(
            scene = {
                'xaxis':  {'title': c1, 'showline': True, 'zeroline': False, 'tickmode': 'auto', 'nticks': 4},
                'yaxis': {'title': c2, 'showline': True, 'zeroline': False, 'tickmode': 'auto', 'nticks': 4},
                'zaxis': {'title': c3, 'showline': True, 'zeroline': False, 'tickmode': 'auto', 'nticks': 4}
            },
        )
    
    layout.update(base_layout)
    return {'data': data, 'layout': layout}

show_app(app)

 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
/home/carlos/anaconda3/envs/data/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://code.jquery.com/jquery-3.2.1.min.js is not available

/home/carlos/anaconda3/envs/data/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js is not available

/home/carlos/anaconda3/envs/data/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of http://getbootstrap.com/dist/js/bootstrap.min.js is not available

/home/carlos/anaconda3/envs/data/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://fonts.googleapis.com/css?family=Raleway:400,300,600 is not available

/home/carlos/anaconda3/envs/data/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://max